In [185]:
import json 
import re
from bs4 import BeautifulSoup
import requests

state_dict = {"ca":"California","sc":"South Carolina"} #Set up dictionary that matches state initials to state name
sport_set = {"basketball","baseball","football"} #Set up sports set 
redundant_words = {"maxpreps","http","https","www","com"}
DEV = 1

relevant_words = {}

####FUNCTIONAL FOR TEAMS IF SPECIFIC URL PROVIDED####
#IN USE NOW
def find_node(html,specific_string = ""):
    if specific_string == "":
        print("NOT YET READY")
        return 0
        #wip
        #TODO
        
    target_text = html.find(string=specific_string)
    print(target_text)
    #returns navigable string so we gotta go to parent node
    #can set up here with predetermined syntax via json lookup or guess. go with json for now.
    #set up error handling
    return target_text.find_parent()

def grab_links(page_elem):
    tags = []
    curr_elem = page_elem
    while len(tags)==0 and curr_elem.name not in {"body", "html", "[document]"}: 
        search_set = [curr_elem] + curr_elem.find_next_siblings()
        for item in search_set:
            found_links = item.find_all('a') #find all anchor tags 
            if found_links:
                tags.extend(found_links) 
        curr_elem = curr_elem.parent
    links = [tag['href'] for tag in tags if 'href' in tag.attrs] #find links
    return links

    
#IN USE NOW 
def grab_team_links(teams_node):
    links = grab_links(teams_node)

    ####REWORK vvvvv
    team_links = [link for link in links if link.startswith("/ca/") and link.endswith("/basketball/") and link != "/ca/basketball/"]
    ####HARDCODED RN but can use relevant words from parse_url_context in the future to decipher relevant links
    return team_links

def grab_roster_links(roster_element): #eventually should make respective classes that override this function
    links = grab_links(roster_element)
    relevant_pattern = r'careerid=[a-z0-9]+' #eventually, it should read off json to get a regex depending on site
    player_links = [link for link in links if re.search(relevant_pattern,link)]
    return player_links

def get_html(URL): #gets html in text
    page = requests.get(URL) 
    soup = BeautifulSoup(page.text,'html.parser') #take html
    return soup

#WIP
def parse_url_context(URL, redundant_words): # remove maxpreps and https:// etc but keep keywords like ca
    # sc, Basketball, Boys to use and find relevant themes. This way, we can remove redundant words from links within page that may point to itself
    #can use these words and additional context to generate likelihood of teams div
    #not use right now
    #chop up url
    url_words = [w for w in re.split(r'[/\.:]+',URL) if w.lower() not in redundant_words and len(w) > 0]
    return url_words

#WIP might not need
def get_team_URL(page_html):
    #try to guess which link is the teams URL on a given site
    return

In [186]:
laf_ball_roster_html = get_html("https://www.maxpreps.com/ca/lafayette/acalanes-dons/basketball/roster/")

In [187]:
team_page_html = get_html("https://www.maxpreps.com/ca/basketball/schools/") 

In [ ]:
##IF WANT TO TEST TEAMS, THERE ARE 200 TEAMS 

In [188]:
roster_element = find_node(laf_ball_roster_html,"Acalanes  Basketball Roster") #FOR SOME REASON THEY HAVE A SPACE which throws off search.
#develop context based title finding 
member_links = grab_roster_links(roster_element)
print(f'{len(member_links)} players\n{member_links}') #as of rn, there are 13 players on this team


Acalanes  Basketball Roster
13 players
['/ca/lafayette/acalanes-dons/athletes/preston-hilsabeck/?careerid=hpb7k1kir1dm6', '/ca/lafayette/acalanes-dons/athletes/cameron-hood/?careerid=dsg8ao87v6ep9', '/ca/lafayette/acalanes-dons/athletes/julian-hood/?careerid=56mf96hmdg7o8', '/ca/lafayette/acalanes-dons/athletes/justice-hembrador/?careerid=2kfkn0i2o19bb', '/ca/lafayette/acalanes-dons/athletes/sam-phillips/?careerid=qd4gfkvejm577', '/ca/lafayette/acalanes-dons/athletes/bryce-mansour/?careerid=hblf6vc03u0bb', '/ca/lafayette/acalanes-dons/athletes/aj-hastings/?careerid=5s34l10a9ftee', '/ca/lafayette/acalanes-dons/athletes/evan-palmer/?careerid=e40dlmenhoaaa', '/ca/lafayette/acalanes-dons/athletes/shea-stahl/?careerid=d8rohq05aqrvf', '/ca/lafayette/acalanes-dons/athletes/jon-macleod/?careerid=23f38c690m8td', '/ca/lafayette/acalanes-dons/athletes/gavin-dodge/?careerid=2eorqhnhgcb44', '/ca/lafayette/acalanes-dons/athletes/aakash-agarwal/?careerid=9u5jbbjt91tm6', '/ca/lafayette/acalanes-dons/a

In [189]:
####PLAYER LOGIC FROM ROSTER#####

#given a team, find the roster link <-- skip since we hardcode it rn

#TODO: from rosters page, find player links

def grab_player_links(roster_element):
    links = grab_links(roster_element)

roster_elem = find_node(laf_ball_roster_html,"Acalanes  Basketball Roster")

Acalanes  Basketball Roster


In [ ]:
##TODO HERE: grab teams. then, from each team, slap that url onto maxpreps and look for roster

In [ ]:
#TODO tentative
    #continue on roster directory logic -- SOMEWHAT DONE
    #continue on player page logic
    #make it create json file that maps teams(from DB) to links
        #"data_pipeline_shortcuts":{
        #"team_url_pattern": "",
        #"team_page_url_pattern": "",
        #"roster_page_url_pattern": "",
        #"player_page_url_pattern": ""
        #},
    #make known path for specific website and add to relevant words

In [ ]:
##EVERYTHING BELOW IS JUST WIP 

In [184]:
redundant_words = {"maxpreps","http","https","www","com"} #could also be relevant words when looking for links that match page context 
URL = "https://www.maxpreps.com/ca/basketball/schools/"
url_words = [w for w in re.split(r'[/\.:]+',URL) if w.lower() not in redundant_words and len(w) > 0]
url_words

['ca', 'basketball', 'schools']

In [ ]:
target_text = soup.find_all(['h1','h2','h3'])
target_text[-1].get_text()
target_text #WIP on pinpointing exact div-- for the far future